In [ ]:
# pip install aesara

In [ ]:
# pip install arviz

In [ ]:
# pip install pymc

In [ ]:
# pip install xarray

In [1]:
import os

import aesara.tensor as at
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import xarray as xr

print(f"Running on PyMC v{pm.__version__}")

WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running on PyMC v4.1.4


In [2]:
RANDOM_SEED = 8924
az.style.use("arviz-darkgrid")

In [3]:
# Import radon data
try:
    srrs2 = pd.read_csv(os.path.join("..", "data", "srrs2.dat"))
except FileNotFoundError:
    srrs2 = pd.read_csv(pm.get_data("srrs2.dat"))

srrs2.columns = srrs2.columns.map(str.strip)
srrs_mn = srrs2[srrs2.state == "MN"].copy()

In [11]:
srrs_mn

,idnum,state,state2,stfips,zip,region,typebldg,floor,room,basement,...,startdt,stopdt,activity,pcterr,adjwt,dupflag,zipflag,cntyfips,county,fips
5080,5081,MN,MN,27,55735,5,1,1,3,N,...,12088,12288,2.2,9.7,1146.499190,1,0,1,AITKIN,27001
5081,5082,MN,MN,27,55748,5,1,0,4,Y,...,11888,12088,2.2,14.5,471.366223,0,0,1,AITKIN,27001
5082,5083,MN,MN,27,55748,5,1,0,4,Y,...,20288,21188,2.9,9.6,433.316718,0,0,1,AITKIN,27001
5083,5084,MN,MN,27,56469,5,1,0,4,Y,...,122987,123187,1.0,24.3,461.623670,0,0,1,AITKIN,27001
5084,5085,MN,MN,27,55011,3,1,0,4,Y,...,12888,13088,3.1,13.8,433.316718,0,0,3,ANOKA,27003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,5995,MN,MN,27,55363,5,1,0,4,Y,...,122687,122887,6.4,4.5,1146.499190,0,0,171,WRIGHT,27171
5995,5996,MN,MN,27,55376,5,1,0,7,Y,...,121787,121987,4.5,8.3,1105.956867,0,0,171,WRIGHT,27171
5996,5997,MN,MN,27,55376,5,1,0,4,Y,...,12888,13088,5.0,5.2,1214.922779,0,0,171,WRIGHT,27171
5997,5998,MN,MN,27,56297,5,1,0,4,Y,...,122887,123087,3.7,9.6,1177.377355,0,0,173,YELLOW MEDICINE,27173


In [4]:
# 그런 다음 두 변수를 결합하여 카운티 수준 예측 변수인 우라늄을 구합니다.
srrs_mn["fips"] = srrs_mn.stfips * 1000 + srrs_mn.cntyfips
cty = pd.read_csv(pm.get_data("cty.dat"))
cty_mn = cty[cty.st == "MN"].copy()
cty_mn["fips"] = 1000 * cty_mn.stfips + cty_mn.ctfips

In [ ]:
srrs_mn = srrs_mn.merge(cty_mn[["fips", "Uppm"]], on="fips")
srrs_mn = srrs_mn.drop_duplicates(subset="idnum")
u = np.log(srrs_mn.Uppm).unique()

n = len(srrs_mn)

In [ ]:
srrs_mn.head()

In [ ]:
srrs_mn.county = srrs_mn.county.map(str.strip)
mn_counties = srrs_mn.county.unique()
counties = len(mn_counties)
county_lookup = dict(zip(mn_counties, range(counties)))

In [ ]:
county_lookup

In [ ]:
county = srrs_mn["county_code"] = srrs_mn.county.replace(county_lookup).values
radon = srrs_mn.activity
srrs_mn["log_radon"] = log_radon = np.log(radon + 0.1).values
floor = srrs_mn.floor.values

In [ ]:
srrs_mn.log_radon.hist(bins=25);

In [ ]:
coords = {"Level": ["Basement", "Floor"]}
with pm.Model(coords=coords) as pooled_model:
    floor_idx = pm.Data("floor_idx", floor, dims="obs_id", mutable=True)
    a = pm.Normal("a", 0.0, sigma=10.0, dims="Level")

    theta = a[floor_idx]
    sigma = pm.Exponential("sigma", 1.0)

    y = pm.Normal("y", theta, sigma=sigma, observed=log_radon, dims="obs_id")

pm.model_to_graphviz(pooled_model)

In [ ]:
with pooled_model:
    prior_checks = pm.sample_prior_predictive()

_, ax = plt.subplots()
prior_checks.prior.plot.scatter(x="Level", y="a", color="k", alpha=0.2, ax=ax)
ax.set_ylabel("Mean log radon level");

In [ ]:
with pooled_model:
    pooled_trace = pm.sample()

pooled_trace.extend(prior_checks)
az.summary(pooled_trace, round_to=2)